<a href="https://colab.research.google.com/github/corioIA/Projet_IA/blob/main/IA_bat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROJET D'IA

I- Introduction

Dans la continuité du cours du premier semestre "Introduction à l'IA et à la data science" nous avons du réalisé un projet de maching learning supervisé dans le but de concevoir, développer et presenter un modèle d"apprentissage automatique. Notre projet consite à prévoir la durée de vie résiduelle des batteries lithium-ion. Nous avions pour cela un esemble de données non-traitées à notre disposition. Vous trouverez dans ce rapport notre démarche expliquée ainsi que notre reflexion. En plus du texte, les lignes de code seront commentées et font parties du rapport.  

II- Prise en main du Data set

Dans un premier temps, nous devions nous familiariser avec le Dataset. Pour cela nous devions d'abord trouver où notre document python était pour mettre au même endroit notre dataset.

In [1]:
pwd

'/content'

La commande dans la cellule ci-dessus permet de savoir où est situé notre document python dans colab et ainsi, nous permet de télécharger le data set au même endroit que notre code pour pouvoir l'ouvrir.
La premiere cellule d'un projet contient toujours les bibliothèques que nous avons utilisées pour le projet. Comme vous pouvez le voir ci-dessous les principales sont :
- Numpy permettant d'effectuer des calculs logiques et mathématiques sur des tableaux et des matrices,
- Pandas permettant une structure de données efficace et rapide pour la manipulation des données avec indexation intégrée,
- Matplotlib et Seaborn permettant la creation de differents graphiques
- Sklearn permettant l'importation du PCA et de différents modèles pour notre projet.







In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV



In [3]:
df=pd.read_csv('Battery_RUL.csv')#Permet la lecture du fichier csv à l'aide de la fonction read de pandas et le tansforme directement en datframe

df.head(10)#Permet d'afficher les 10 permieres ligne du dataframe

FileNotFoundError: [Errno 2] No such file or directory: 'Battery_RUL.csv'

In [ ]:
df.shape# Nous permet d'avoir le nombre de colonne et lignes du dataframe


In [ ]:
df.info()#Nous donne diffèrentes information sur le dataframe telle que le rangeIndex le Datatype ou la mémoire utilisée.

In [ ]:
df.describe()#Permet de décrire le dataframe et perment de nous projeter sur le nettoyage de celui-ci

Après avoir pris en main le dataframe à l'aide de différentes méthodes des différentes bibliothèques, nous avons maintenant une meilleure vue d'ensemble sur nos données. Nous devons maintenant passer à l'étape la plus cruciale : le nettoyage de la base de données.


III- Le Nettoyage et analyse


Le nettoyage d'un data set permet de supprimer les valeurs pouvant dégrader la performance de nos modèles choisis.
Les différents éléments à nettoyer sont les doublons, les valeurs aberrantes et les valeurs manquantes.

Nous avons choisi de commencer par la suppression des valeurs aberrantes. Pour cela, nous avons effectué une analyse descriptive sur chacune de nos variables à l'aide de la methode boxplot de la bibliothèque seaborn.


In [ ]:
#Analyse descriptive de la variable Cycle_Index
sns.boxplot(data=df, y="Cycle_Index")


La variable "Cycle index" ne requiert aucun nettoyage, nous n'avons pas detecté de valeurs aberrantes. La deuxième variable "Discharge time (s)" nécessite un nettoyage. En effet, nous remarquons que celle-ci contient des valeurs négatives. Or, un temps ne peut être négatif. Après cela nous nous sommes renseignés sur le temps de décharge moyen d'une batterie lithium-ion. Nous avons remarqué que comparer à cette moyenne les valeurs max et min étaient beaucoup trop divergentes. Nous avons donc décidé de ne garder que les valeur au dessus de 1150 secondes et en dessous de 2500 secondes. Effectivement, l'utilisateur n'utilisera plus sa batterie à partir de certainnes performaces. Il n'attendra pas par exemple que celle-ci dure seulement 5 secondes pour la changer, d'où notre encadrement assez élevé.

In [ ]:
df=df[df['Discharge Time (s)']>=1150]
df=df[df['Discharge Time (s)']<=2500]
df.describe()

In [ ]:
#Analyse descriptive de la variable Discharge Time
sns.boxplot(data=df, y="Discharge Time (s)")

De même pour la variable "Decrement 3.6-3.4V (s)" nous remarquons des valeurs de temps négatives que nous supprimons également.

In [ ]:
#Analyse descriptive de la variable Decrement 3.6-3.4V
sns.boxplot(data=df, y="Decrement 3.6-3.4V (s)")

In [ ]:
#Suppression des valuers abérantes.
df=df[df['Decrement 3.6-3.4V (s)']>=0] # on affecte au dataframe toute les valeur qui n'ont pas de Decrement 3.6-3.4V (s) < 0 ce qui supprime ainsi toute les valeurs negative. Un temps négatif n'existant pas
#df.describe()

In [ ]:
#Analyse descriptive de la variable Max. Voltage Dischar.
sns.boxplot(data=df, y="Max. Voltage Dischar. (V)")

La variable "Max. Voltage Dischar. (V)" et La variable "Min. Voltage Charg. (V)" ne semble pas avoir de valeurs aberrantes. Nous avons donc décidé de ne rien supprimer pour ces deux variables.

In [ ]:
#Analyse descriptive de la variable Min. Voltage Charg.
sns.boxplot(data=df, y="Min. Voltage Charg. (V)")

In [ ]:
#Analyse descriptive de la variable Time at 4.15V
sns.boxplot(data=df, y="Time at 4.15V (s)")

La variable 'Time at 4.15V (s)' contient  aussi des valeurs de temps négatives que nous supprimons.

In [ ]:
#Suppression des valeurs abérantes de la colonne Time at 4.15V (s).
df=df[df['Time at 4.15V (s)']>=0]
sns.boxplot(data=df, y="Time at 4.15V (s)")# On replot la box pour voir les effets de nos modifications



Nous avons décidé de ne pas toucher à la variable "Time constant current (s)" qui semble correcte.

In [ ]:
#Analyse descriptive de la variable Time constant current
sns.boxplot(data=df, y="Time constant current (s)")

La variable "Charging time (s)" contenait elle aussi des valeurs de temps négatives. Et comme pour la variable "Discharge Time (s)", nous avons decidé de créer un encadrement plus précis pour pouvoir garder que des valeurs améliorant notre data set et ainsi pouvoir mieux prédire la duréé de vie résiduelle d'une batterie. Nous nous sommes donc renseignés sur le temps de charge moyen d'une batterie lithium-ion et nous avons trouvé que les valeurs maximum et minimum étaient soit beaucoup trop hautes soit beaucoup trop faibles. Par exemple la valeur minimal après avoir supprimé les valeurs négatives était de 5 secondes ce qui en toute logique est beaucoup trop faible pour recharger une batterie. Après reflexion, nous sommes donc arrivé à l'encadrement dans la cellule ci-dessous qui est entre 7200 seconde et 9750 seconde.

In [ ]:
#Suppression des valeurs abérantes.
df=df[df['Charging time (s)']>=7200]
df=df[df['Charging time (s)']<=9750]

#df.describe()

In [ ]:
#Analyse descriptive de la variable Charging time (s).
sns.boxplot(data=df, y="Charging time (s)")

La variable "RUL" n'a pas eu besoin d'être nettoyé.

In [ ]:
#Analyse descriptive de la variable RUL
sns.boxplot(data=df, y="RUL")

Dans la cellule ci-dessous la méthode duplicate nous permet de trouver les doublons présents dans le data set. Et, comme indiqué dans la cellule, notre data set ne contient aucun doublon.

In [ ]:
#Recherche de doublon dans le dataSet
df.duplicated().sum() # Il n'y a pas de doublon

Nous devions ensuite créer une matrice de corrélation. Celle- ci permet comme son nom l'indique de voir la corrélation entre  les diffèrentes variables de notre data set. Si les valeurs sont entre 0,7 et 1 valeur alors il y a une forte corrélation entre les deux variables et entre 0 et 0.7 ont avec une mauvaise corrélation entre les valeurs. Idem pour les valeurs négatives. Pour créer la matrice nous avons utilisé la bibliothèque seaborn qui nous explique à l'aide de sa documentaton comment la réaliser. Les commentaires dans la cellule ci-dessous explique les différentes étapes de création.

In [ ]:
#Matrice de corrélation
corr = df.corr()

# Génere un masque pour le triangle supérieur
mask = np.triu(np.ones_like(corr, dtype=bool))

# Crée un graphe matplotlib
f, ax = plt.subplots(figsize=(11, 9))

# Geénère une carte de couleur divergente
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Dessine la heatmap avec un mask et un aspect ration corrigé
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, annot=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Nous avons ensuite tracé des courbes bi-variable. Pour pouvoir tirer des résultats, nous avons tracé des valeurs fortement corrélées pour pouvoir tirer des info des courbes et pouvant nous donné des intuitions pour le feature engineering. Le RUL étant notre target et la Variable "Cycle Index" n'étant pas utile dans nos modèles nous ne traçons pas de courbes avec ces deux variables. Plus blabla sur les courbes et rajouter des courbes

In [ ]:
#Courbe bivariable

sns.scatterplot(data=df,x='Discharge Time (s)',y='Time at 4.15V (s)')



In [ ]:
sns.scatterplot(data=df,x='Discharge Time (s)',y='Decrement 3.6-3.4V (s)')

In [ ]:
sns.scatterplot(data=df,x='Time at 4.15V (s)',y='Time constant current (s)')

In [ ]:
sns.scatterplot(data=df,x='Discharge Time (s)',y='Charging time (s)')

In [ ]:
sns.scatterplot(data=df,x='Discharge Time (s)',y='Time constant current (s)')

III- Feature Engineering

Le feature engineering consiste à créer de nouvelles variables nous permettant de rendre plus efficace et plus précis notre ensemble d'étude et ainsi obtenir un modèle d'une meilleure qualité. Pour cela il faut trouver des liens entre les différentes variables descriptives de notre système pour effectuer, entre elles des opérations mathématiques pour en obtenir de nouvelles.
Dans certains cas, il est possible de ne pas faire de feature engineering. Ici, après l'étude de la matrice de corrélation et des graphes bi-variés, nous observons une relation de proportionnalité (courbe linéaire) entre Discharge time et Time at 4.15V ainsi que Discharge time et Time constant current. Sachant que la variable Discharge Time est un indicateur de la santé de notre batterie (plus il diminue plus la batterie présente des signes de faiblesse) et, étant fortement corrélé avec les deux autres variables citées précédemment, il nous a paru juste de créer nos deux features présentent dans la cellule ci-dessous.
Le ratio créé indique la tendance suivie par nos variables, cela ajoute donc une information complémentaire importante quant à la compréhension de notre dataset.

In [ ]:
#Feature engineering
#Ratio discharge/time at 4.15 V
#Ratio discharge/time at constant courant
df['Ratio_Discharge_4.15V']=df['Discharge Time (s)']/df['Time at 4.15V (s)']
df['Ratio_Discharge_Constant_Current']=df['Discharge Time (s)']/df['Time constant current (s)']
df.drop('Cycle_Index',axis=1,inplace=True)# La variable Cycle index ne sert pas pour la modélisation.
df.head()



In [ ]:
rul=df['RUL']
df.drop('RUL',axis=1,inplace=True)
rul.shape

Dans la cellule ci-dessus, nous séparons notre target de notre ensemble de données. Il est nécessaire de faire cela au préalable pour pas que notre target entre dans les variables que notre modèle devra traiter pour prédire un résultat.

Dans la cellule ci-dessous, nous créons notre dataset d'entrainement et notre dataset test ainsi que ceux de la target. L'ensemble d'entrainement est composé à 80% de nos données. Les autres 20% étant pour le test.

In [ ]:
# création du data set d'entrainement et du data set de test

df_train, df_test, rul_train, rul_test = train_test_split(df,rul,train_size=0.8 ,test_size=0.2, random_state=42)

Pour la suite de notre feature engineering, il faut normaliser les variables du dataset.
Nous n'avons pas normalisé la target car cela n'entrait pas dans notre champ de compétences. Mais il est d'usage de normaliser celle-ci.

In [ ]:
#normalisation
scaler = skl.preprocessing.StandardScaler() # Create a StandardScaler instance
scaled_data_train = scaler.fit_transform(df_train) # Fit and transform the data
scaled_data_test = scaler.fit_transform(df_test)

Dans la case ci-dessous, nous allons procéder à l'ACP. L'objectif principal de l'ACP est de transformer un jeu de données avec de nombreuses variables en un jeu de données avec moins de variables tout en préservant autant que possible l'information présente dans les données originales. Cela permet de rendre l'analyse plus rapide et plus facile, sans perdre trop d'informations pertinentes. Ceci correspond par exemple à zipper un dossier.
L'ACP se fait à l'aide de la bibliothèque sklearn importée précedemment.
L'un des problème rencontré a été de créer notre ACP avec la méthode .predict qui n'existe pas dans cette bibliothèque. En réaction, nous avons utilisé la méthode .transform. Pour le dataset d'entrainement, il faut utiliser la méthode .fit_transform.

In [ ]:

#df_test_ACP=ACP.predict(df_scaled_test)

def perform_pca_analysis(X, n_components=0.95):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)

    plt.figure(figsize=(15, 5))

    # Variance expliquée cumulée
    plt.subplot(1, 2, 1)
    plt.plot(np.cumsum(pca.explained_variance_ratio_), 'bo-')
    plt.xlabel('Nombre de composantes')
    plt.ylabel('Variance expliquée cumulée')
    plt.title('Variance expliquée cumulée par composante')
    plt.grid(True)

    # Scree plot
    plt.subplot(1, 2, 2)
    plt.plot(pca.explained_variance_ratio_, 'ro-')
    plt.xlabel('Composantes')
    plt.ylabel('Variance expliquée individuelle')
    plt.title('Scree Plot')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    return pca, X_pca


pca, train_pca = perform_pca_analysis(scaled_data_train )


train_pca.shape




In [ ]:
test_pca= pca.transform(scaled_data_test)#ensemble de données du test aprés tranformation ACP
test_pca.shape

IV- Moélisation et évaluation


Lors de notre modélisation, nous avons réalisé 3 modèles :
 - Une régression linéaire
 - Une random forest
 - Une gradient boost

 Pour évaluer les performances de nos modèles, nous avons choisi d'utiliser les métriques d'évaluation :      
  - MAE (mesure l'écart moyen entre les valeurs réelles et les valeurs prédites, sans tenir compte du signe de l'erreur)
  - MSE (mesure la différence entre les valeurs réelles et les valeurs prédites par le modèle)
  - RMSE (est la racine carrée de la MSE)
  - r2 (mesure la fiabilité de notre modèle)

  Nous ferons maintenant une analyse de chaque modèles et de leurs metriques.

In [ ]:
#modeles
model=LinearRegression()
model.fit(train_pca, rul_train)
rul_pred=model.predict(test_pca)
mae=mean_absolute_error(rul_test,rul_pred)
mse=mean_squared_error(rul_test,rul_pred)
rmse=np.sqrt(mse)
r2=r2_score(rul_test,rul_pred)
print(mae)
print(mse)
print(rmse)
print(r2)





Pour prendre en main la modélisations de modèle, nous avons choisi de commencé par une regression linéaire.

Ici, notre MAE est de 36.24. Ce qui signifie que, en moyenne, notre modèle fait une erreur de 36.24 cycles dans ses prédictions de durée de vie de la batterie. Cela donne une idée directe de l'écart moyen entre les prédictions et les valeurs réelles.

De plus, une MSE de 2342.10 suggère que notre modèle a des erreurs importantes (en termes de carré des écarts) qui, par rapport à d'autres modèles ou approches, pourraient être considérées comme assez grandes. Cependant, bien que la MSE soit plus grande que la MAE, ce n'est pas nécessairement un problème, car elle est plus sensible aux grandes erreurs. Cela peut indiquer que notre modèle a quelques prédictions qui sont bien en dehors de la réalité.

En outre, le RMSE est simplement la racine carrée de la MSE, ce qui le ramène à l'échelle des données d'origine. Cela donne une idée de l'erreur moyenne en termes d'unités de la variable cible. Un RMSE de 48.40, indique que l'écart-type des erreurs est d'environ 48.40 cycles.

Pour finir, un r2 de 0,96 nous indique que notre modèle est fiable à 96%. Il est donc très performant.




In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(train_pca, rul_train)
rul_pred = model.predict(test_pca)
mae=mean_absolute_error(rul_test,rul_pred)
mse=mean_squared_error(rul_test,rul_pred)
rmse=np.sqrt(mse)
r2=r2_score(rul_test,rul_pred)
print(mae)
print(mse)
print(rmse)
print(r2)


 La random forest consiste en un ensemble d'arbres de décision qui travaillent ensemble pour fournir des prédictions plus robustes et plus précises que les arbres de décision individuels.
 La Random Forest est une méthode d'ensemble, ce qui signifie qu'elle combine plusieurs modèles pour améliorer la performance globale. Dans le cas du Random Forest, ce sont des arbres de décision qui sont combinés pour former un "forêt" de décisions.
Le modèle utilise des hyperparamètres. Nous expliqueront dans la partie V ce qu'est un hyperparamètre et comment nous les avons optimiser.
Dans un prmeir temps les hyperamètres selectionnés sont ceux par défaut.

Nous obtenons les métriques suivant:
- Une MAE de 20,7, ce qui signifie que notre système se trompe de 20,7 cycles de durée de vie en moyenne
- Une MSE de 1264 nous indique que notre modèle possède des erreurs assez élevées en moyenne et qu'il est sensible aux grandes erreurs.
- Une RMSE de 35,5 cycles signifie que nous avons un écart-type des erreurs qui est en moyenne de 35,5 cycles.
- Enfin, une r2 de 0,98 nous montre quze notre modèle est fiable à 98% et qu'il possède donc une grande performance.

Comparé au modèle de régression linéaire ce modèle est plus performant.


In [ ]:
model = GradientBoostingRegressor(n_estimators=100, random_state=42)
model.fit(train_pca, rul_train)
rul_pred = model.predict(test_pca)
mae=mean_absolute_error(rul_test,rul_pred)
mse=mean_squared_error(rul_test,rul_pred)
rmse=np.sqrt(mse)
r2=r2_score(rul_test,rul_pred)
# Affichage des résultats
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

Pour le 3ème modèle, nous avons choisi de faire un gradient boosting regressor. Cet estimateur construit un modèle additif de manière progressive, étape par étape ; il permet d'optimiser des fonctions de perte différentiables arbitraires. À chaque étape, un arbre de régression est ajusté sur le gradient négatif de la fonction de perte donnée.
Tout come le modèle précédent, celui-ci utilise des hyperparamètres défini par défaut.
Nous obtenons les métriques suivant:
- Une MAE de 29,8, ce qui signifie que notre système se trompe de 29,8 cycles de durée de vie en moyenne
- Une MSE de 1705,6 nous indique que notre modèle possède des erreurs assez élevées en moyenne et qu'il est sensible aux grandes erreurs.
- Une RMSE de 41,3 cycles signifie que nous avons un écart-type des erreurs qui est en moyenne de 41,3 cycles.
- Enfin, une r2 de 0,97 nous montre quze notre modèle est fiable à 98% et qu'il possède donc une grande performance.

Nous obtenons des résultats similaires à notre model random forest. Bien que ceux-ci soient moins bons.


Dans leur globalité, nos trois modèles obtiennent des résultats satisfaisant bien que nous ayons une MSE assez élevée. Ce métrique peut être amélioré grâce à l'optimisation des hyperparamètres.
Cela traduit notre bonne compréhension du sujet et notre bon traitement du dataset.

V-Hyperparamètre


Les hyperparamètres sont des paramètres que l'utilisateur doit spécifier et qui influencent la façon dont le modèle va apprendre à partir des données.
Nous choississons de faire cette optimisation sur le modèle random forest (Celui-ci étant notre modèle le plus performant).

De plus, lors de l'optimisation des hyperparamètres, il est crucial d'utiliser la validation croisée pour éviter le sur-apprentissage. Cela permet de s’assurer que le modèle est bien évalué sur différentes portions de données et qu’il peut généraliser de manière fiable sur des données non vues.

Dans scikit-learn, l'option cv dans GridSearchCV et RandomizedSearchCV effectue automatiquement la validation croisée. Nous en faisons 3.

Nous choisissons d'optimiser les paramètres suivant:

- n_estimators, le nombre d'arbre
- max_depth, la profondeur de la fôret
- min_samples_split, le nombre minimum d'echantillon permettant de spérare un noeud interne
- min_samples_leaf, le minimum d'échatillon pour former un feuille
- max_feature, le nombre de feature à considérer pour avoir la meilleure séparation.





In [ ]:
model=RandomForestRegressor()

param_dist={'n_estimators': np.arange(100, 500, 150), "max_depth" : [5,10,15,None], "min_samples_split" : [2, 5, 10], "min_samples_leaf" : [1, 2, 4], 'max_features': ['auto', 'sqrt', 'log2']}

model_opti=RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=75, cv=3, n_jobs=-1)
model_opti.fit(train_pca, rul_train)
print(model_opti.best_params_)

In [ ]:
model = RandomForestRegressor(n_estimators=400, min_samples_split=2,min_samples_leaf=1, max_features='sqrt' ,random_state=42)
model.fit(train_pca, rul_train)
rul_pred = model.predict(test_pca)
mae=mean_absolute_error(rul_test,rul_pred)
mse=mean_squared_error(rul_test,rul_pred)
rmse=np.sqrt(mse)
r2=r2_score(rul_test,rul_pred)
print(mae)
print(mse)
print(rmse)
print(r2)

L'optimisation des hyperparmètres étant trés couteuse en temps nous avons optimiser que certains paramètres et fait un nombre faible d'itération. L'optimisation nous donnant des valeurs de métriques trés proche de celle obtenues sans optimisation, nous concluons que notre optimisation n'est pas satisfaisante et qu'il faudrait soit plus de temps pour tester plus de combinaisons soit une plus grande puissance de calcul sachant que nous utilisons déja tout les coeurs disponiblent. Cependant, nous arrivons quand même à diminuer la MSE.

VI- Analyse des résultats

Pour ce projet d'IA, nous devions créer un programme à partir d'un dataset pour estimer la durée de vie résiduel d'une batterie lithium-ion.
Nous avons procédé en suivant les étapes d'un projet d'IA à savoir :
- Nettoyage du dataset
- feature engineering
- Préparation du dataset (test et entrainement)
- Modélisation
- Optimisation
- Evaluation
Au cours de l'étape de compréhension des données, nous avons trouvé que les variables les plus importantes pour avoir une bonne estimation étaient :    
- Discharge Time
- Time at 4.15V
- Time at constant current

Ensuite, nous avons fait le choix de trois modèles différents :    
- Regression
- Random forest regression
- Grandient boost

En évaluant ces trois modèles nous avons déterminer que le meilleur était le random forest regression. Nous avons ensuite procédé à une optimisation de ses hyperparamètres.
Nous obtenons une évaluation très pertinentes avec par exemple un r2 de 0,98 ce qui reflète bien la fiabilité de notre modèle. De plus notre MAE est de 22,4 ce qui signifie que notre modèle se trompe de 22,4 cycle. Le nombre de cycle oscillant autour de 1000 cycles, nous avons donc une erreur de seulement 2%. Notre modèle possède donc de bonnes performances et est donc viable pour le calcul de la durée de vie résiduelle d'une batterie.


VII-Conclusion


Pour conclure, notre modèle présente de bonnes qualités. Il peut néanmoins être amélioré en utilisant d'autres types de modèles que les trois utilisés lors de ce projet. Par exemple, l'utilisation d'un réseau de neuronne qui au vu des thèses faites sur le sujet est plus répendu. De plus nous pourrons également utilisé une autre technioque pour l'optimisation des hyperparamètres. Nous pourrons également améliorer la partie feature engineering en rajoutant des features qui apporteraient plus d'informations à notre dataset.